# Importing libaries

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

# Importing first databases

Radars.csv contains all cars, trucks, motorcycles and buses that comes thru São Paulo's radar system 


In [3]:

a = pd.read_csv(r"D:\\Users\\guilh\\Documents\\GitHub\\Dados_CET\\Marco_2018_nAg\\2_nAg.csv", usecols= ["Data", "Numero Agrupado", "Faixa", "Registro", "Especie", "Classe", "Velocidade"], index_col= "Data")


## Reading the columns

* **Radar** is the number of identification of a street section
* **Lane** goes from 1 to 6 in most radars, low lane number are closer to the center of the freeway, high lane numbers are "local" lanes, to the right
* **Register** represents each vehicle
* **Types** are: motorcycle = 0, car = 1, bus = 2 ou truck = 3
* **Classes** are: *light* (motorcycle and car) = 0 ou *heavy* (bus and truck)  = 1
* **Speeds** are in kilometer per hour


In [10]:
# Preprocessing
df = a.sample(frac=0.1)
# turns speed from dm/s to km/h
df["Velocidade"] = df["Velocidade"] * 0.36

df.index.names = ["Date"]

df["Radar_Lane"] = df["Numero Agrupado"] + df["Faixa"]

# renaming columns to english
df.columns = ["Radar", "Lane", "Register", "Type", "Class", "Speed [km/h]", "Radar_Lane"]

df.head()

MemoryError: 

In [ ]:
lane_types = pd.read_excel(r"D:\Users\guilh\Documents\[POLI]_6_Semestre\IC\2021\codigos olimpio\\Faixa Tipo.xlsx", usecols = ["Num_agrupado","faixa", "Num_fx","tipo"])


In [ ]:
lane_types.head()